## GBDT

### GBDT 回归

In [ ]:
import numpy as np


# 获取 训练数据

dataset = np.array(
               [[1, 5, 20, 1.1],
                [2, 7, 30, 1.3],
                [3, 21, 70, 1.7],
                [4, 30, 60, 1.8],
               ])
columns=['id', 'age', 'weight', 'label']


X = dataset[:,0:3]
y = dataset[:,3]

np.mean(y)


In [ ]:
import numpy as np 

In [ ]:

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import ensemble
 
 
# 加载sklearn自带的波士顿房价数据集
dataset = load_boston()
 
# 提取特征数据和目标数据
X = dataset.data
y = dataset.target
 
# 将数据集以9:1的比例随机分为训练集和测试集，为了重现随机分配设置随机种子，即random_state参数
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, test_size=0.1, random_state=188)
 
# 实例化估计器对象
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
gbr = ensemble.GradientBoostingRegressor(**params)
 
# 估计器拟合训练数据
gbr.fit(X_train, y_train)
 
# 训练完的估计器对测试数据进行预测
y_pred = gbr.predict(X_test)
 
# 输出特征重要性列表
print(gbr.feature_importances_)
print(mean_squared_error(y_test, y_pred))

In [ ]:
np.shape(X)

In [ ]:
def sigmoid(  X):
    """
    sigmoid 激活函数
    :param X:
    :return:
    """
    return 1 / (1 + np.exp(-X))

sigmoid(float('-inf'))

### GBDT 二分类

#### GDBT_2Classifier 实现

In [ ]:
import numpy as np

import time

from sklearn.ensemble import GradientBoostingClassifier

from sklearn import datasets

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import ensemble


import sys

sys.path.append('E:\\python package\\python-project\\统计学习方法\\Statistical-Learning-Method_Code-master\\DecisionTree')

from CartTree_regression_xrh import *



In [ ]:
class GDBT_2Classifier:
    """

    适用于 二分类 问题 的 梯度提升树

    基分类器为 CART 回归树

    Author: xrh
    Date: 2021-04-10

    ref: https://zhuanlan.zhihu.com/p/89549390


    test1: 二分类任务

    数据集：Mnist
    参数: error_rate_threshold=0.01, max_iter=30, max_depth=3,learning_rate=0.2

    训练集数量：60000
    测试集数量：10000
    正确率： 0.9891
    模型训练时长：205.7052161693573

    """

    def __init__(self, error_rate_threshold=0.05, max_iter=10, max_depth=1):
        """

        :param error_rate_threshold: 训练中止条件, 若当前得到的基分类器的组合 的错误率 小于阈值, 则停止训练
        :param max_iter: 最大迭代次数
        :param max_depth: CART 回归树 的最大深度
        """

        # 训练中止条件 error_rate  < self.error_rate_threshold ( 若当前得到的基分类器的组合 的错误率 小于阈值, 则停止训练)
        self.error_rate_threshold = error_rate_threshold

        # 最大迭代次数
        self.max_iter = max_iter

        # CART 回归树 的最大深度
        self.max_depth = max_depth

        self.G = []  # 弱分类器 集合

    def sigmoid( self , X ):
        """
        sigmoid 激活函数
        :param X:
        :return:
        """
        return 1 / (1 + np.exp(-X))

    def fit(self, X, y, learning_rate):
        """

        用 训练数据 拟合模型

        :param X: 特征数据 , shape=(N_sample, N_feature)
        :param y: 标签数据 , shape=(N_sample,)
        :param learning_rate: 学习率
        :return:
        """


        N = np.shape(X)[0]  # 样本的个数

        f = 0  # 基分类器 的加权和

        P_1 = len(y[y == 1]) / len(y)
        y_predict = np.log(P_1 / (1 - P_1))

        self.G.append(y_predict)

        f += y_predict

        feature_value_set = RegresionTree.get_feature_value_set(X)  # 可供选择的特征集合 , 包括 (特征, 切分值)

        for m in range(self.max_iter):  # 进行 第 m 轮迭代

            r = y - self.sigmoid(f)  # 残差

            RT = RegresionTree_GBDT(min_square_loss=0.1, max_depth=self.max_depth,print_log=False)

            RT.fit(X, r, y, feature_value_set=feature_value_set)

            y_predict = RT.predict(X)

            self.G.append((learning_rate, RT))  # 存储 基分类器

            # 计算 当前 所有弱分类器加权 得到的 最终分类器 的 分类错误率

            f += learning_rate * y_predict

            G = self.sigmoid(f)

            #TODO: 负例 设置为 -1 会导致 在训练集的 训练误差率无法下降, 原因未知

            G[G >= 0.5] = 1  # 概率 大于 0.5 被标记为 正例
            G[G < 0.5] = 0  # 概率 小于 0.5 被标记为 负例

            err_arr = np.ones(N, dtype=int)
            err_arr[G == y] = 0
            err_rate = np.mean(err_arr)

            print('round:{}, err_rate:{}'.format(m, err_rate))
            print('======================')

            if err_rate < self.error_rate_threshold:  # 错误率 已经小于 阈值, 则停止训练
                break

    def predict(self, X):
        """
        对 测试 数据进行预测, 返回预测的标签

        :param X: 特征数据 , shape=(N_sample, N_feature)
        :return:
        """

        f = 0  # 最终分类器

        f += self.G[0]  # 第一个 存储的是 初始化情况

        for alpha, RT in self.G[1:]:
            y_predict = RT.predict(X)
            f += alpha * y_predict

        # print('f:',f)

        G = self.sigmoid(f)

        # print('G:',G)

        G[G >= 0.5] = 1  # 概率 大于 0.5 被标记为 正例
        G[G < 0.5] = 0  # 概率 小于 0.5 被标记为 负例

        return G

    def predict_proba(self, X):
        """
        对 测试 数据进行预测, 返回预测的 概率值

        :param X: 特征数据 , shape=(N_sample, N_feature)
        :return:
        """

        f = 0  # 最终分类器

        f += self.G[0]  # 第一个 存储的是 初始化情况

        for alpha, RT in self.G[1:]:
            y_predict = RT.predict(X)
            f += alpha * y_predict

        # print('f:',f)
        G = self.sigmoid(f)

        return G


    def score(self, X, y):
        """
        使用 测试数据集 对模型进行评价, 返回正确率

        :param X: 特征数据 , shape=(N_sample, N_feature)
        :param y: 标签数据 , shape=(N_sample,)
        :return:  正确率 accuracy
        """

        N = np.shape(X)[0]  # 样本的个数

        G = self.predict(X)

        err_arr = np.ones(N, dtype=int)
        err_arr[G == y] = 0
        err_rate = np.mean(err_arr)

        accuracy = 1 - err_rate

        return accuracy

In [ ]:

def loadData_2classification( fileName, n=1000):
    '''
    加载文件

    将 数据集 的标签 转换为 二分类的标签

    :param fileName:要加载的文件路径
    :param n: 返回的数据集的规模
    :return: 数据集和标签集
    '''
    # 存放数据及标记
    dataArr = []
    labelArr = []
    # 读取文件
    fr = open(fileName)

    cnt = 0  # 计数器

    # 遍历文件中的每一行
    for line in fr.readlines():

        if cnt == n:
            break

        # 获取当前行，并按“，”切割成字段放入列表中
        # strip：去掉每行字符串首尾指定的字符（默认空格或换行符）
        # split：按照指定的字符将字符串切割成每个字段，返回列表形式
        curLine = line.strip().split(',')
        # 将每行中除标记外的数据放入数据集中（curLine[0]为标记信息）
        # 在放入的同时将原先字符串形式的数据转换为整型
        # 此外将数据进行了二值化处理，大于128的转换成1，小于的转换成0，方便后续计算
        dataArr.append([int(int(num) > 128) for num in curLine[1:]])

        # 将标记信息放入标记集中
        # 转换成二分类任务
        # 标签0设置为1，反之为0

        # 显然这会导致 正负 样本的 分布不均衡, 1 的样本很少(10%), 而0 的很多
        if int(curLine[0]) == 0:
            labelArr.append(1)
        else:
            labelArr.append(0)

        # if int(curLine[0]) <= 5:
        #     labelArr.append(1)
        # else:
        #     labelArr.append(0)

        cnt += 1

    fr.close()

    # 返回数据集和标记
    return dataArr, labelArr
    
    

In [ ]:
n_train=6000

# 获取训练集
trainDataList, trainLabelList =loadData_2classification('../Mnist/mnist_train.csv', n=n_train)

print('train data, row num:{} , column num:{} '.format(len(trainDataList), len(trainDataList[0])))

trainDataArr = np.array(trainDataList)
trainLabelArr = np.array(trainLabelList)


In [ ]:

# 开始时间
print('start training model....')
start = time.time()



clf = GDBT_2Classifier( error_rate_threshold=0.01, max_iter=30, max_depth=3 )
clf.fit(trainDataArr, trainLabelArr,learning_rate=0.2)


# 结束时间
end = time.time()
print('training cost time :', end - start)




In [ ]:

clf2 = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=30
                                  , max_depth=3
                                )
clf2.fit(trainDataArr, trainLabelArr)



In [ ]:
n_test=1000

# 获取测试集
testDataList, testLabelList = loadData_2classification('../Mnist/mnist_test.csv', n=n_test)

print('test data, row num:{} , column num:{} '.format(len(testDataList), len(testDataList[0])))

testDataArr = np.array(testDataList)
testLabelArr = np.array(testLabelList)

print('test dataset accuracy: {} '.format(clf.score(testDataArr, testLabelArr)))

#### 模型评估 

In [ ]:
from sklearn.metrics import confusion_matrix 
from sklearn.utils.class_weight import compute_sample_weight

from sklearn.metrics import accuracy_score

from sklearn.metrics import precision_score

from sklearn.metrics import recall_score

from sklearn.metrics import f1_score


y_pred= clf.predict( testDataArr )
y_true=testLabelArr


#0. 混淆矩阵


print('confusion_matrix: ')
print(confusion_matrix(y_true, y_pred)) # 主对角线上的 元素为 标签值 和 预测值 都为 第 i 类 的样本的数目

# array([[ 83,   0,   0,   0,   0,   1,   1,   0,   0,   0],
#        [  0, 124,   0,   1,   0,   0,   1,   0,   0,   0],
#        [  0,   1, 104,   1,   0,   0,   1,   3,   5,   1],
#        [  0,   0,   1,  96,   0,   5,   1,   1,   2,   1],
#        [  1,   0,   0,   0, 100,   0,   1,   1,   1,   6],
#        [  0,   0,   0,   3,   1,  79,   0,   3,   1,   0],
#        [  3,   0,   0,   0,   1,   1,  82,   0,   0,   0],
#        [  0,   0,   2,   2,   2,   1,   0,  87,   1,   4],
#        [  0,   0,   1,   4,   3,   2,   0,   2,  77,   0],
#        [  0,   0,   0,   0,   2,   0,   0,   3,   1,  88]], dtype=int64)

# confusion_matrix[0][0]= 83  真实标签为 0 预测值也为0  的样本个数为 83 
# confusion_matrix[2][8]= 5    真实标签为 2 预测值为8  的样本个数为 5

#样本不均衡时,  改变样本的权重 
sw = compute_sample_weight(class_weight='balanced',y=y_true)

confusion_matrix(y_true, y_pred, sample_weight=sw)

# array([[97.64705882,  0.        ,  0.        ,  0.        ,  0.        ,
#          1.17647059,  1.17647059,  0.        ,  0.        ,  0.        ],
#        [ 0.        , 98.41269841,  0.        ,  0.79365079,  0.        ,
#          0.        ,  0.79365079,  0.        ,  0.        ,  0.        ],
#        [ 0.        ,  0.86206897, 89.65517241,  0.86206897,  0.        ,
#          0.        ,  0.86206897,  2.5862069 ,  4.31034483,  0.86206897],
#        [ 0.        ,  0.        ,  0.93457944, 89.71962617,  0.        ,
#          4.6728972 ,  0.93457944,  0.93457944,  1.86915888,  0.93457944],
#        [ 0.90909091,  0.        ,  0.        ,  0.        , 90.90909091,
#          0.        ,  0.90909091,  0.90909091,  0.90909091,  5.45454545],
#        [ 0.        ,  0.        ,  0.        ,  3.44827586,  1.14942529,
#         90.8045977 ,  0.        ,  3.44827586,  1.14942529,  0.        ],
#        [ 3.44827586,  0.        ,  0.        ,  0.        ,  1.14942529,
#          1.14942529, 94.25287356,  0.        ,  0.        ,  0.        ],
#        [ 0.        ,  0.        ,  2.02020202,  2.02020202,  2.02020202,
#          1.01010101,  0.        , 87.87878788,  1.01010101,  4.04040404],
#        [ 0.        ,  0.        ,  1.12359551,  4.49438202,  3.37078652,
#          2.24719101,  0.        ,  2.24719101, 86.51685393,  0.        ],
#        [ 0.        ,  0.        ,  0.        ,  0.        ,  2.12765957,
#          0.        ,  0.        ,  3.19148936,  1.06382979, 93.61702128]])


print('====================')

# 1.正确率
print('test dataset accuracy: {} '.format(accuracy_score(y_true, y_pred)))

print('====================')

# 2.精确率  

# print(precision_score(y_true, y_pred, average='macro'))  # 
# print(precision_score(y_true, y_pred, average='micro'))  # 
# print(precision_score(y_true, y_pred, average='weighted'))  # 

print('pos-1 precision: ',precision_score(y_true, y_pred, average='binary')) 

precision_list= precision_score(y_true, y_pred, average=None)

print('neg-0 precision:{}, pos-1 precision:{}  '.format(precision_list[0],precision_list[1]) ) 

print('====================')

# 3. 召回率

# print(recall_score(y_true, y_pred, average='macro'))  # 
# print(recall_score(y_true, y_pred, average='micro'))  # 
# print(recall_score(y_true, y_pred, average='weighted'))  # 

print('pos-1 recall: ',recall_score(y_true, y_pred, average='binary')) 

recall_list= recall_score(y_true, y_pred, average=None)

print('neg-0 recall:{}, pos-1 recall:{}  '.format(recall_list[0],recall_list[1]) )

print('====================')

# 4. F1-score

# print(f1_score(y_true, y_pred, average='macro'))  
# print(f1_score(y_true, y_pred, average='micro')) 
# print(f1_score(y_true, y_pred, average='weighted'))  

print('pos-1 f1_score: ',f1_score(y_true, y_pred, average='binary')) 

f1_score_list= f1_score(y_true, y_pred, average=None)

print('neg-0 f1_score:{}, pos-1 f1_score:{}  '.format(f1_score_list[0],f1_score_list[1]) )

print('====================')



In [ ]:
y_scores2= clf2.predict_proba(testDataArr)[:,0]
# y_scores2

precision2, recall2, thresholds2 = precision_recall_curve(y_true, y_scores2)

##### P-R 曲线

In [ ]:
# 画出 P-R 曲线

from itertools import cycle

# colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal'])

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

# from sklearn.metrics import PrecisionRecallDisplay


y_pred= clf.predict(testDataArr)

y_scores= clf.predict_proba(testDataArr)

y_true=testLabelArr



precision, recall, thresholds = precision_recall_curve(y_true, y_scores)


y_scores2= clf2.predict_proba(testDataArr)[:,1]  # 第 1 列 , 表示为 正例的概率

precision2, recall2, thresholds2 = precision_recall_curve(y_true, y_scores2)



# disp = PrecisionRecallDisplay(precision=precision, recall=recall)
# disp.plot() 


plt.plot(recall, precision, label="GDBT_2Classifier(xrh)",color='navy' ) # 

plt.plot(recall2, precision2, label="GradientBoostingClassifier(sklearn)",color='turquoise' )


plt.title(' Precision-Recall curve ')

# plt.ylim([0.0, 1.05]) # Y 轴的取值范围
# plt.xlim([0.0, 1.0]) # X 轴的取值范围



plt.xlabel("recall")
plt.ylabel("precision")

plt.legend( loc=(0, -.38), prop=dict(size=14) ) # 图例

plt.show()



**P-R 关系图的 左右端点的讨论**

In [ ]:
ratio_pos= len(y_true[y_true==1]) / len(y_true)
ratio_pos

In [ ]:
# 左端点
recall2[-1], precision2[-1],thresholds2[-1]

In [ ]:
# 右端点
recall2[0], precision2[0],thresholds2[0]

##### ROC 曲线

In [ ]:
from sklearn.metrics import roc_curve


fpr, tpr, _ = roc_curve(y_true, y_scores)

fpr2, tpr2, _ = roc_curve(y_true, y_scores2)

plt.plot( [0, 1], [0, 1], color='navy', linestyle='--' )


plt.plot( fpr, tpr, label="GDBT_2Classifier(xrh)",color='darkorange' ) # 

plt.plot( fpr2, tpr2, label="GradientBoostingClassifier(sklearn)",color='turquoise' )

# plt.xlim( [0.0, 1.0] )
# plt.ylim( [0.0, 1.05] )

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')


plt.legend( loc=(0, -.38), prop=dict(size=14) ) # 图例

plt.show()

**ROC 曲线的 左右端点的讨论**

In [ ]:
# 右端点
fpr[-1], tpr[-1],thresholds[-1]

In [ ]:
# 左端点
fpr[0], fpr[0],thresholds2[0]

### GBDT 多分类

In [ ]:


X_train=[  [6],
           [12],
           [14],
           [18],
           [20],
           [65],
           [31],
           [40],
           [1],
           [2],
           [100],
           [101],
           [65],
           [54],
           ]

y_train= np.array([[0], [0], [0], [0], [0], [1], [1], [1], [1], [1], [2], [2], [2], [2]]).ravel()

X=X_train

y=y_train

max_iter=5

N = np.shape(X)[0]  # 样本的个数



K = len({ele for ele in y})  # y 中有多少种不同的标签,  K分类

print('according to the training dataset : K={} classification task'.format(K))

F = np.zeros( (K , N),dtype=float) # shape: (K,N)

for k in range(K): # 遍历 所有的 类别

    F[k,:] = len(y[y == k]) / len(y)

    


In [ ]:
def softmax(X):
    """
    softmax处理，将结果转化为概率

    :param X:
    :return:
    """
    return np.e ** X / ((np.e ** X).sum(axis=0))  # softmax处理，将结果转化为概率


In [ ]:

from scipy.special import expit, logsumexp


raw_predictions=F

np.nan_to_num( np.exp(raw_predictions - logsumexp(raw_predictions, axis=0)) )


softmax( F )



In [ ]:
m=1

p = softmax( F )



y_one_hot = ( y == np.array(range(K)).reshape(-1, 1) ).astype(
    np.int8)  # 将 预测向量 扩展为 one-hot , shape: (K,N)

p_k=p[0]
y_one_hot_k = y_one_hot[0]

r=y_one_hot_k - p_k

r

In [ ]:

F_0 = np.array( [1,2,3],dtype=float)  # shape : (K,)
    
np.transpose([F_0] * 3)

In [ ]:
np.exp(3085)

####  GBDT_MultiClassifier 实现


In [ ]:
import numpy as np

import time

from sklearn.ensemble import GradientBoostingClassifier

from sklearn import datasets

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import ensemble


import sys

sys.path.append('E:\\python package\\python-project\\统计学习方法\\Statistical-Learning-Method_Code-master\\DecisionTree')

from CartTree_regression_xrh import *

from scipy.special import expit, logsumexp


In [ ]:
class GBDT_MultiClassifier:
    """

    适用于 二分类 问题 的 梯度提升树

    基分类器为 CART 回归树

    Author: xrh
    Date: 2021-04-18

    ref: https://zhuanlan.zhihu.com/p/91652813

    test1: 多分类任务

    数据集：Mnist
    参数: error_rate_threshold=0.01, max_iter=20, max_depth=3 , learning_rate=0.5
    训练集数量：60000
    测试集数量：10000
    正确率： 0.915
    模型训练时长：1542s

    """

    def __init__(self, error_rate_threshold=0.05, max_iter=10, max_depth=1):
        """

        :param error_rate_threshold: 训练中止条件, 若当前得到的基分类器的组合 的错误率 小于阈值, 则停止训练
        :param max_iter: 最大迭代次数
        :param max_depth: CART 回归树 的最大深度
        """

        # 训练中止条件 error_rate  < self.error_rate_threshold ( 若当前得到的基分类器的组合 的错误率 小于阈值, 则停止训练)
        self.error_rate_threshold = error_rate_threshold

        # 最大迭代次数
        self.max_iter = max_iter

        # CART 回归树 的最大深度
        self.max_depth = max_depth

        self.G = []  # 弱分类器 集合

    def sigmoid( self , X ):
        """
        sigmoid 激活函数
        :param X:
        :return:
        """
        return 1 / (1 + np.exp(-X))

    # def softmax_deprecated(self,X):
    #     """
    #     softmax处理，将结果转化为概率
    #
    #     :param X:
    #     :return:
    #     """
    #     #TODO: 导致 上溢出 和 下溢出 问题
    #
    #     return  np.exp(X) / np.sum( np.exp(X) , axis=0 )  # softmax处理，将结果转化为概率

    def softmax_deprecated(self,X):
        """
        softmax处理，将结果转化为概率

        解决了 softmax的 上溢出 和 下溢出的问题

        ref: https://www.cnblogs.com/guoyaohua/p/8900683.html

        :param X: shape (K,N)
        :return: shape (N,)
        """

        X_max= np.max( X, axis=0)
        X= X-X_max

        return  np.exp(X) / np.sum( np.exp(X) , axis=0 )  # softmax处理，将结果转化为概率

    def softmax(self,X):
        """
        softmax处理，将结果转化为概率

        解决了 softmax的 溢出问题

        np.nan_to_num : 使用0代替数组x中的nan元素，使用有限的数字代替inf元素

        ref: sklearn 源码
            MultinomialDeviance -> def negative_gradient

        :param X: shape (K,N)
        :return: shape (N,)
        """

        return  np.nan_to_num( np.exp(X - logsumexp(X, axis=0)) )  # softmax处理，将结果转化为概率

    def fit(self, X, y, learning_rate):
        """

        用 训练数据 拟合模型

        :param X: 特征数据 , shape=(N_sample, N_feature)
        :param y: 标签数据 , shape=(N_sample,)
        :param learning_rate: 学习率
        :return:
        """

        N = np.shape(X)[0]  # 样本的个数

        self.K = len({ele for ele in y})  # y 中有多少种不同的标签,  K分类

        print('according to the training dataset : K={} classification task'.format(self.K))

        F_0 = np.zeros( (self.K ),dtype=float)  # shape : (K,)

        for k in range(self.K): # 遍历 所有的 类别

            F_0[k] = len(y[y == k]) / len(y)

        self.G.append(F_0)

        F = np.transpose([F_0] * N) # 对 F_0 进行复制,  shape : (K, N)

        feature_value_set = RegresionTree.get_feature_value_set(X)  # 可供选择的特征集合 , 包括 (特征, 切分值)

        y_one_hot = (y == np.array(range(self.K)).reshape(-1, 1)).astype(
            np.int8)  # 将 预测向量 扩展为 one-hot , shape: (K,N)

        for m in range(1,self.max_iter):  # 进行 第 m 轮迭代

            p = self.softmax( F ) #  shape: (K,N)

            DT_list=[]

            for k in range(self.K): #  依次训练 K 个 二分类器

                print( '======= train No.{} 2Classifier ======='.format(k) )

                r = y_one_hot[k] - p[k]   # 残差 shape:(N,)

                # 训练 用于 2分类的 回归树
                DT = RegresionTree_GBDT(min_square_loss=0.1, max_depth=self.max_depth,print_log=False)

                DT.fit(X, r, y_one_hot[k], feature_value_set=feature_value_set)

                y_predict = (self.K / (self.K-1)) * ( DT.predict(X) ) #  shape:(N,)

                DT_list.append(DT)

                F[k] += learning_rate * y_predict  # F[k]  shape:(N,)

                # print('======= end =======')


            self.G.append( (learning_rate, DT_list) )  # 存储 基分类器

            # 计算 当前 所有弱分类器加权 得到的 最终分类器 的 分类错误率

            G = self.softmax( F )

            G_label = np.argmax( G, axis=0 )  # 取 概率最大的 作为 预测的标签

            err_arr = np.ones( N, dtype=int )
            err_arr[G_label == y] = 0
            err_rate = np.mean(err_arr)  # 计算训练误差

            print('round:{}, err_rate:{}'.format(m, err_rate))
            print('======================')

            if err_rate < self.error_rate_threshold:  # 错误率 已经小于 阈值, 则停止训练
                break

    def predict(self, X):
        """
        对 测试 数据进行预测, 返回预测的标签

        :param X: 特征数据 , shape=(N_sample, N_feature)
        :return:
        """
        N = np.shape(X)[0]  # 样本的个数

        F_0 = self.G[0]  # G中 第一个 存储的是 初始化情况

        F = np.transpose([F_0] * N)  # shape : (K, N)

        for alpha, DT_list in self.G[1:]:

            for k in range(self.K):

                DT = DT_list[k]

                y_predict = (self.K / (self.K - 1)) * (DT.predict(X))  # shape:(N,)

                F[k] += alpha * y_predict  # F[k]  shape:(N,)

        G = self.softmax(F)

        G_label = np.argmax(G, axis=0)

        return G_label

    def predict_proba(self, X):
        """
        对 测试 数据进行预测, 返回预测的 概率值

        :param X: 特征数据 , shape=(N_sample, N_feature)
        :return:
        """

        F = self.G[0]  # 第一个 存储的是 初始化情况

        for alpha, DT_list in self.G[1:]:

            for k in range(self.K):
                DT = DT_list[k]

                y_predict = (self.K / (self.K - 1)) * (DT.predict(X))  # shape:(N,)

                DT_list.append(DT)

                F[k] += alpha * y_predict  # F[k]  shape:(N,)

        G = self.softmax(F)

        return G


    def score(self, X, y):
        """
        使用 测试数据集 对模型进行评价, 返回正确率

        :param X: 特征数据 , shape=(N_sample, N_feature)
        :param y: 标签数据 , shape=(N_sample,)
        :return:  正确率 accuracy
        """

        N = np.shape(X)[0]  # 样本的个数

        G = self.predict(X)

        err_arr = np.ones(N, dtype=int)
        err_arr[G == y] = 0
        err_rate = np.mean(err_arr)

        accuracy = 1 - err_rate

        return accuracy

In [ ]:
 class Test:   
    
    def loadData(self, fileName, n=1000):
        '''
        加载文件

        :param fileName:要加载的文件路径
        :param n: 返回的数据集的规模
        :return: 数据集和标签集
        '''
        # 存放数据及标记
        dataArr = []
        labelArr = []
        # 读取文件
        fr = open(fileName)

        cnt = 0  # 计数器

        # 遍历文件中的每一行
        for line in fr.readlines():


            if cnt == n:
                break

            # 获取当前行，并按“，”切割成字段放入列表中
            # strip：去掉每行字符串首尾指定的字符（默认空格或换行符）
            # split：按照指定的字符将字符串切割成每个字段，返回列表形式
            curLine = line.strip().split(',')
            # 将每行中除标记外的数据放入数据集中（curLine[0]为标记信息）
            # 在放入的同时将原先字符串形式的数据转换为整型
            # 此外将数据进行了二值化处理，大于128的转换成1，小于的转换成0，方便后续计算
            dataArr.append([int(int(num) > 128) for num in curLine[1:]])

            # 将标记信息放入标记集中
            labelArr.append(int(curLine[0]))
            cnt += 1

        fr.close()

        # 返回数据集和标记
        return dataArr, labelArr

    def test_Mnist_dataset(self, n_train, n_test):
        """
         Mnist (手写数字) 数据集

        测试 AdaBoost  的 多分类

        :param n_train: 使用训练数据集的规模
        :param n_test: 使用测试数据集的规模
        :return:
        """

        # 获取训练集
        trainDataList, trainLabelList = self.loadData('../Mnist/mnist_train.csv', n=n_train)

        print('train data, row num:{} , column num:{} '.format(len(trainDataList), len(trainDataList[0])))

        trainDataArr = np.array(trainDataList)
        trainLabelArr = np.array(trainLabelList)

        # 开始时间
        print('start training model....')
        start = time.time()

        """
        调参：
        loss：损失函数。有deviance和exponential两种。deviance是采用对数似然，exponential是指数损失，后者相当于AdaBoost。
        n_estimators:最大弱学习器个数，默认是100，调参时要注意过拟合或欠拟合，一般和learning_rate一起考虑。
        criterion: 切分叶子节点时, 选择切分特征考虑的误差函数, 默认是 “ friedman_mse”（ Friedman 均方误差），“ mse”（均方误差）和“ mae”（均绝对误差）
        learning_rate:步长，即每个弱学习器的权重缩减系数，默认为0.1，取值范围0-1，当取值为1时，相当于权重不缩减。较小的learning_rate相当于更多的迭代次数。
        subsample:子采样，默认为1，取值范围(0,1]，当取值为1时，相当于没有采样。小于1时，即进行采样，按比例采样得到的样本去构建弱学习器。这样做可以防止过拟合，但是值不能太低，会造成高方差。
        init：初始化弱学习器。不使用的话就是第一轮迭代构建的弱学习器.如果没有先验的话就可以不用管
        由于GBDT使用CART回归决策树。以下参数用于调优弱学习器，主要都是为了防止过拟合
        max_feature：树分裂时考虑的最大特征数，默认为None，也就是考虑所有特征。可以取值有：log2,auto,sqrt
        max_depth：CART最大深度，默认为None
        min_sample_split：划分节点时需要保留的样本数。当某节点的样本数小于某个值时，就当做叶子节点，不允许再分裂。默认是2
        min_sample_leaf：叶子节点最少样本数。如果某个叶子节点数量少于某个值，会同它的兄弟节点一起被剪枝。默认是1
        min_weight_fraction_leaf：叶子节点最小的样本权重和。如果小于某个值，会同它的兄弟节点一起被剪枝。一般用于权重变化的样本。默认是0
        min_leaf_nodes：最大叶子节点数
        
        ref: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
        
        测试1: 
        max_depth=3, n_estimators=30, learning_rate=0.8, 
        n_train=60000
        n_test=10000
        训练时间 : 795.5719292163849
        准确率: 0.8883 
        
        测试2:
        max_depth=3, n_estimators=20, learning_rate=0.5, 
        n_train=60000
        n_test=10000
        训练时间 : 589 s
        准确率: 0.9197 
        
        """

        clf = GradientBoostingClassifier(loss='deviance',criterion='mse', n_estimators=20, learning_rate=0.5,
                                         max_depth=3)

        clf.fit(trainDataArr, trainLabelArr)



        # clf = GBDT_MultiClassifier( error_rate_threshold=0.01, max_iter=20, max_depth=3 )
        # clf.fit( trainDataArr, trainLabelArr,learning_rate= 0.5 ) #

        # 结束时间
        end = time.time()
        print('training cost time :', end - start)

        # 获取测试集
        testDataList, testLabelList = self.loadData('../Mnist/mnist_test.csv', n=n_test)

        print('test data, row num:{} , column num:{} '.format(len(testDataList), len(testDataList[0])))

        testDataArr = np.array(testDataList)
        testLabelArr = np.array(testLabelList)

        print('test dataset accuracy: {} '.format(clf.score(testDataArr, testLabelArr)))

In [ ]:
test = Test()

test.test_Mnist_dataset(6000,1000)

In [ ]:
def loadData( fileName, n=1000):
        '''
        加载文件

        :param fileName:要加载的文件路径
        :param n: 返回的数据集的规模
        :return: 数据集和标签集
        '''
        # 存放数据及标记
        dataArr = []
        labelArr = []
        # 读取文件
        fr = open(fileName)

        cnt = 0  # 计数器

        # 遍历文件中的每一行
        for line in fr.readlines():


            if cnt == n:
                break

            # 获取当前行，并按“，”切割成字段放入列表中
            # strip：去掉每行字符串首尾指定的字符（默认空格或换行符）
            # split：按照指定的字符将字符串切割成每个字段，返回列表形式
            curLine = line.strip().split(',')
            # 将每行中除标记外的数据放入数据集中（curLine[0]为标记信息）
            # 在放入的同时将原先字符串形式的数据转换为整型
            # 此外将数据进行了二值化处理，大于128的转换成1，小于的转换成0，方便后续计算
            dataArr.append([int(int(num) > 128) for num in curLine[1:]])

            # 将标记信息放入标记集中
            labelArr.append(int(curLine[0]))
            cnt += 1

        fr.close()

        # 返回数据集和标记
        return dataArr, labelArr

    
    


In [ ]:


n_train=60000
n_test=10000

# 获取训练集
trainDataList, trainLabelList = loadData('../Mnist/mnist_train.csv', n=n_train)

print('train data, row num:{} , column num:{} '.format(len(trainDataList), len(trainDataList[0])))

trainDataArr = np.array(trainDataList)
trainLabelArr = np.array(trainLabelList)

# 开始时间
print('start training model....')
start = time.time()



# clf = GradientBoostingClassifier(loss='deviance',criterion='mse', n_estimators=40, learning_rate=0.5,
#                                  max_depth=3)

# clf.fit(trainDataArr, trainLabelArr)


clf = GBDT_MultiClassifier( error_rate_threshold=0.01, max_iter=70, max_depth=4 )
clf.fit( trainDataArr, trainLabelArr,learning_rate= 0.5 ) #

# 结束时间
end = time.time()
print('training cost time :', end - start)

# 获取测试集
testDataList, testLabelList = loadData('../Mnist/mnist_test.csv', n=n_test)

print('test data, row num:{} , column num:{} '.format(len(testDataList), len(testDataList[0])))

testDataArr = np.array(testDataList)
testLabelArr = np.array(testLabelList)

print('test dataset accuracy: {} '.format(clf.score(testDataArr, testLabelArr)))


sklearn 调参 

loss：损失函数。有deviance和exponential两种。deviance是采用对数似然，exponential是指数损失，后者相当于AdaBoost。

n_estimators:最大弱学习器个数，默认是100，调参时要注意过拟合或欠拟合，一般和learning_rate一起考虑。

criterion: 切分叶子节点时, 选择切分特征考虑的误差函数, 默认是 “ friedman_mse”（ Friedman 均方误差），“ mse”（均方误差）和“ mae”（均绝对误差）

learning_rate:步长，即每个弱学习器的权重缩减系数，默认为0.1，取值范围0-1，当取值为1时，相当于权重不缩减。较小的learning_rate相当于更多的迭代次数。

subsample:子采样，默认为1，取值范围(0,1]，当取值为1时，相当于没有采样。小于1时，即进行采样，按比例采样得到的样本去构建弱学习器。这样做可以防止过拟合，但是值不能太低，会造成高方差。

init：初始化弱学习器。不使用的话就是第一轮迭代构建的弱学习器.如果没有先验的话就可以不用管

由于GBDT使用CART回归决策树。以下参数用于调优弱学习器，主要都是为了防止过拟合

max_feature：树分裂时考虑的最大特征数，默认为None，也就是考虑所有特征。可以取值有：log2,auto,sqrt

max_depth：CART最大深度，默认为None

min_sample_split：划分节点时需要保留的样本数。当某节点的样本数小于某个值时，就当做叶子节点，不允许再分裂。默认是2

min_sample_leaf：叶子节点最少样本数。如果某个叶子节点数量少于某个值，会同它的兄弟节点一起被剪枝。默认是1

min_weight_fraction_leaf：叶子节点最小的样本权重和。如果小于某个值，会同它的兄弟节点一起被剪枝。一般用于权重变化的样本。默认是0

min_leaf_nodes：最大叶子节点数


ref: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

测试1: 
max_depth=3, n_estimators=30, learning_rate=0.8, 
n_train=60000
n_test=10000
训练时间 : 795.5719292163849
准确率: 0.8883 

测试2:
max_depth=3, n_estimators=20, learning_rate=0.5, 
n_train=60000
n_test=10000
训练时间 : 589 s
准确率: 0.9197 


#### 学习曲线

In [ ]:
from xgboost import XGBRegressor as XGBR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.linear_model import LinearRegression as LinearR
from sklearn.datasets import load_boston
from sklearn.model_selection import KFold, cross_val_score as CVS, train_test_split as TTS
from sklearn.metrics import mean_squared_error as MSE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time
import datetime

In [ ]:

#来查看一下sklearn中所有的 模型评估指标
import sklearn
sorted(sklearn.metrics.SCORERS.keys())


In [ ]:
n_train=6000
n_test=1000

# 获取训练集
trainDataList, trainLabelList = loadData('../Mnist/mnist_train.csv', n=n_train)

In [ ]:
trainDataArr = np.array(trainDataList)
trainLabelArr = np.array(trainLabelList)

Xtrain,Ytrain = trainDataArr, trainLabelArr 

In [ ]:
def plot_learning_curve(estimator,title, X, y, 
                        ax=None, #选择子图
                        ylim=None, #设置纵坐标的取值范围
                        cv=None, #交叉验证
                        n_jobs=None #设定索要使用的线程
                       ):
    
    from sklearn.model_selection import learning_curve
    import matplotlib.pyplot as plt
    import numpy as np
    
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y
                                                            ,shuffle=True
                                                            ,cv=cv
                                                            ,random_state=420
                                                            ,n_jobs=n_jobs)      
    if ax == None:
        ax = plt.gca()
    else:
        ax = plt.figure()
    ax.set_title(title)
    if ylim is not None:
        ax.set_ylim(*ylim)
    ax.set_xlabel("Training examples")
    ax.set_ylabel("Score")
    ax.grid() #绘制网格，不是必须
    ax.plot(train_sizes, np.mean(train_scores, axis=1), 'o-'
            , color="r",label="Training score")
    ax.plot(train_sizes, np.mean(test_scores, axis=1), 'o-'
            , color="g",label="Test score")
    ax.legend(loc="best")
    return ax


In [ ]:
cv = KFold(n_splits=5, shuffle = True, random_state=42) # k 折 交叉验证

KFold ref:

https://blog.csdn.net/kancy110/article/details/74910185

In [ ]:
for train_index , test_index in cv.split(Xtrain):
    print('train_index:%s , test_index: %s ' %(train_index,test_index))
          

In [ ]:
plot_learning_curve(GradientBoostingClassifier(loss='deviance',criterion='mse', n_estimators=20, learning_rate=0.5,
                                 max_depth=3)
                    ,"GBDT",Xtrain,Ytrain,ax=None,cv=cv)
plt.show()

#### 交叉验证

In [ ]:
clf = GradientBoostingClassifier(loss='deviance',criterion='mse', n_estimators=20, learning_rate=0.5,
                                 max_depth=3)

CVS( clf, Xtrain,Ytrain ,cv=5,scoring='accuracy') #  cv : 交叉验证的 折数

In [ ]:
# gbdt = GBDT_MultiClassifier( error_rate_threshold=0.01, max_iter=20, max_depth=3 )
# CVS( gbdt, Xtrain,Ytrain ,cv=5,scoring='accuracy') #  cv : 交叉验证的 折数

In [ ]:
# 超参数 n_estimators 调优
#=====【TIME WARNING： seconds】=====#

axisx = range(10,100,10)
rs = []
for i in axisx:
    
    reg =  GradientBoostingClassifier(loss='deviance',criterion='mse', n_estimators=i, learning_rate=0.5,
                                 max_depth=3)
    rs.append( CVS(reg,Xtrain,Ytrain,cv=cv).mean() )
    
print( axisx[rs.index(max(rs))], max(rs) ) # n_estimators=80  accuracy=0.9189999999999999

plt.figure(figsize=(20,5))
plt.plot( axisx,rs,c="red",label="GBDT" )
plt.legend()
plt.show()

In [ ]:
# 超参数 learning_rate 调优

axisx = np.linspace(0.5,1,10)
rs = []
for i in axisx:
    
    reg =  GradientBoostingClassifier(loss='deviance',criterion='mse', n_estimators=20, learning_rate=i,
                                 max_depth=3)
    
    rs.append( CVS(reg,Xtrain,Ytrain,cv=5,n_jobs=-1).mean() ) #  n_jobs=-1 使用所有的CPU核, 并行度实际为5, 因为使用 5折交叉验证       ，
    
    
print( axisx[rs.index(max(rs))], max(rs) ) 

plt.figure(figsize=(20,5))
plt.plot( axisx,rs,c="red",label="GBDT" )
plt.legend()
plt.show()

#### 网格搜索

ref:
https://www.cnblogs.com/wj-1314/p/10422159.html

In [ ]:

from sklearn.model_selection import GridSearchCV

#首先我们先来定义一个评分函数，这个评分函数能够帮助我们直接打印Xtrain上的交叉验证结果
def clfassess(clf,Xtrain,Ytrain,cv,scoring = ["accuracy"],show=True):
    
    score = []
    for i in range(len(scoring)):
        
        c=CVS (clf,Xtrain,Ytrain,cv=5,scoring=scoring[i]).mean()
        
        if show:
            print("{}:{:.2f}".format(scoring[i] #模型评估指标的名字
                                ,c))
            
        score.append((c).mean())
        
    return score



def print_best_score(gsearch,param_test):
     # 输出best score
    print("Best score: %0.3f" % gsearch.best_score_)
    print("Best parameters set:")
    # 输出最佳的分类器到底使用了怎样的参数
    best_parameters = gsearch.best_estimator_.get_params()
    for param_name in sorted(param_test.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

        


In [ ]:

param = {
    'n_estimators':  range(20,90,10),
    'max_depth': range(1,5,1)
}
#网格搜索 是 两个 参数集合的全组合(笛卡尔积), 因此 集合中的元素个数 不宜过多

estimator=GradientBoostingClassifier(loss='deviance',criterion='mse', n_estimators=20, learning_rate=0.5,
                                 max_depth=3)

gsearch = GridSearchCV( estimator , param_grid = param, scoring='accuracy', cv=5 , n_jobs=-1 )

gsearch.fit( Xtrain,Ytrain )


print_best_score(gsearch,param)

# Best score: 0.924
# Best parameters set:
# 	max_depth: 4
# 	n_estimators: 70


In [ ]:
gsearch.best_params_